# 4. Análise e Exploração dos Dados


[ml map sklearn](https://scikit-learn.org/stable/tutorial/machine_learning_map/)

In [2]:
# ! pip install seaborn
! pip install plotly
# ! pip install statsmodels
# ! pip install sklearn

     |████████████████████████████████| 26.5 MB 10.7 MB/s eta 0:00:01
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


## importar bibiliotecas

In [3]:
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf


from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif, f_regression

import joblib

## Carregar dados tratados

In [4]:
def carregar_joblib():
    df = joblib.load("../out/trat_target_col_ibov.joblib")
    return df

### Histograma Comprar e vender col decisao

In [12]:
def print_histograma_comprar_vender(df):
    print("\nHistograma Comprar e Vender\n")
    df = df.copy()
    # Muda os nomes para histograma
    df['decisao'].replace({0: "Vender", 1: "Comprar"}, inplace=True)
    # Imprime a porcentagem de cada
    print(df['decisao'].value_counts(normalize=True).map("{:.1%}".format))

    # Imprime o histograma
    fig = px.histogram(df, x="decisao", color="decisao")
    fig.update_layout(bargap=0.2)
    fig.show()


### Correlações

In [6]:
def print_correlacoes(df):

    fig, ax = plt.subplots(figsize=(15, 10))
    sns.heatmap(df.corr(), cmap="Wistia", annot=True, fmt = '.2f', ax=ax)
    plt.show()

## P value

In [7]:
def print_p_value(df):
    all_col = " + ".join(df.drop("decisao", axis=1).columns)
    model = smf.glm(formula="decisao ~ " + all_col, data=df,
                    family = sm.families.Binomial()).fit()

    print(model.summary()) 

## Feature selection

In [8]:
def feat_sel_select_k_best(df):
    print("\nFeature selection select k best\n")

    k = 6  # quantidade de variaveis a serem selecionadas
    
    X = df.drop("decisao", axis=1)
    y = df["decisao"]

    select = SelectKBest(score_func=f_classif, k=k)
    
    select.fit(X, y)

    filter = select.get_support()
    
    print(f"\n{k} melhores variaveis:")
    print(f"{[col for col in X.columns[filter]]} \n")
    
    return list(X.columns[filter])


In [ ]:
def remover_variaveis_ja_explicadas(df, manter_variaveis):
    to_remove = set(manter_variaveis) ^ set(df.columns)
    to_remove.remove('decisao')
    print("Colunas a remover do df original: ", to_remove)
    df = df.drop(to_remove, axis=1)

    return df

## Salvar resultado para proxima estapa

In [10]:
def salvar_df_in_joblib(df, file_name: str = "../out/result_analise_desc.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df analise descritiva criado com sucesso")


# Execução

In [17]:
def main():
#     print("Analise e feature selection notebook\n\n")

    df = carregar_joblib()

    print(f"Carregado df com {df.shape[0]} linhas e {df.shape[1]} colunas.\n")
    print(f"Colunas carregadas: {[col for col in df.columns]}\n\n")

    print_histograma_comprar_vender(df)

    print_correlacoes(df)

    print("#-" * 40)
    best_features = feat_sel_select_k_best(df)
    
    df = remover_variaveis_ja_explicadas(df, manter_variaveis=best_features)

    print("\nCorrelacoes apos remover colunas\n")

    print_correlacoes(df)

    print("Remover coluna resultado_bruto por ter correlacao com receita_liq")

    df = remover_variaveis_ja_explicadas(df, manter_variaveis=['outros_ativos_circ', 'receita_liq', 'despesas_gerais', 'resultado_antes_trib', 'lucro_op'])

    print("\nCorrelacoes apos remover colunas\n")

    print_correlacoes(df)

    print("#-" * 40)

    print_p_value(df)

    salvar_df_in_joblib(df, "../out/result_analise_desc.joblib")

    return df

    
df = main()


Feature selection select k best


6 melhores variaveis:
['outros_ativos_circ', 'receita_liq', 'resultado_bruto', 'despesas_gerais', 'resultado_antes_trib', 'lucro_op'] 

Colunas a remover do df original:  {'caixa_equiv_caixa', 'passivo_circ', 'passivo_total', 'imobilizado', 'ativo_permanente', 'capital_social', 'ativo_circ', 'ativo_total', 'financeiras', 'outros_ativos_nao_circ', 'intangivel', 'custo_bens_servicos', 'patrimonio_liq', 'passivo_nao_circ'}
Colunas a remover do df original:  {'resultado_bruto'}


,outros_ativos_circ,receita_liq,despesas_gerais,resultado_antes_trib,lucro_op,decisao
0,0.073416,-0.009957,-0.178706,0.784128,0.714242,1
1,21.051236,-0.306352,0.322396,-6.964708,-4.169923,0
2,-0.954322,0.109726,0.103272,0.649179,0.216400,0
3,0.088899,-0.407359,0.330099,0.540194,0.746589,1
4,-0.242792,0.633929,-0.603944,-0.245369,-0.156249,1
